In [46]:
import pprint 
import psycopg2 
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT 
from psycopg2.extensions import AsIs 
import os

def pg_con(): 
    con = psycopg2.connect(
            dbname='postgres', 
            user='admin', 
            host='localhost', 
            port='30960', 
            password='password'
    )
    return(con) 

In [47]:
pg_con()

<connection object at 0x10ef13828; dsn: 'user=admin password=xxx dbname=postgres host=localhost port=30960', closed: 0>

In [48]:
c=pg_con().cursor() 

In [53]:
c

<cursor object at 0x1053edce0; closed: 0>

In [62]:
name_Database   = "BostonCrime";
sqlCreateDatabase = "create database "+name_Database+";"
# Create a table in PostgreSQL database

c.execute(sqlCreateDatabase);

InFailedSqlTransaction: current transaction is aborted, commands ignored until end of transaction block


In [49]:
import pandas as pd
df = pd.read_csv('/Users/jasminekaur/Desktop/basf/Code/data/Boston_Crime_Dataset.csv',encoding="latin_1",dtype='unicode' )  

In [50]:
df.dtypes

INCIDENT_NUMBER        object
OFFENSE_CODE           object
OFFENSE_CODE_GROUP     object
OFFENSE_DESCRIPTION    object
DISTRICT               object
REPORTING_AREA         object
OCCURRED_ON_DATE       object
YEAR                   object
MONTH                  object
DAY_OF_WEEK            object
HOUR                   object
UCR_PART               object
STREET                 object
Lat                    object
Long                   object
Location               object
dtype: object

In [59]:
c.execute("CREATE TABLE INCIDENT(INCIDENT_NUMBER text,DISTRICT text, YEAR integar,MONTH integar,DAY_OF_WEEK integar,HOUR integar,OFFENSE_CODE integar,OFFENSE_CODE_GROUP text,OFFENSE_DESCRIPTION text,STREET text);")
con.commit()

InFailedSqlTransaction: current transaction is aborted, commands ignored until end of transaction block


In [61]:
CREATE TABLE Incident(INCIDENT_NUMBER text);

SyntaxError: invalid syntax (<ipython-input-61-ff2462e3f503>, line 1)